In [1]:
#Importing required libraries
import selenium
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product name to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
#Connect to web driver
driver=webdriver.Chrome(r"D://chromedriver.exe")  #r converts string to raw string
#If not r, we can use executable_path = "C:/path name"

In [3]:
#Getting the website to driver
driver.get('https://www.amazon.in/')

#When we run this line, automatically the webpage will be opened

In [4]:
#Searching headphones in the search bar and clicking the search button
search_bar=driver.find_element_by_id('twotabsearchtextbox')
search_bar.send_keys("headphones")

driver.find_element_by_id('nav-search-submit-button').click()

# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [5]:
#Creating a for loop to append the urls of first 3 pages
URL=[]  #Empty list
for page in range(0,3):
    for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        URL.append(i.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break   
        
#Taking the empty lists
brand_name=[]
product_name=[]
Rating=[]
no_of_ratings=[]
price=[]
returns=[]
delivery=[]
availability=[]
other_details=[]
product_url=[]
     
#We are running a for loop for the url as it changes for every page and we are extracting the product url
for i in URL:
    driver.get(i)
    product_url.append(i)
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the brand name
    try:
        brand_name.append(driver.find_element_by_id("productTitle").text.split(' ')[0])
    except NoSuchElementException as e:
        brand_name.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds   
    
    #Extracting the product name
    try:
        string=' '  #Empty string
        for j in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+j  #Extracting the text and appending to empty string
        product_name.append(string)  #Appending the string having the extracted text
    except NoSuchElementException as e:
            product_name.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds     
    
    #Extracting the ratings
    try:
        Rating.append(driver.find_element_by_id("acrPopover").get_attribute('title')) 
    except NoSuchElementException as e:
        Rating.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the number of ratings
    try:
        no_of_ratings.append(driver.find_element_by_id("acrCustomerReviewText").text) 
    except NoSuchElementException as e:
        no_of_ratings.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the price
    try:
        try:
            price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try:
                price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except:
                price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except NoSuchElementException as e:
        price.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting the Return/Exchange
    try:
        returns.append(driver.find_element_by_xpath("//div[@id='RETURNS_POLICY']").text)
    except NoSuchElementException as e:
        returns.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds 
    
    #Extracting Expected Delivery
    try:
        delivery.append(driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b').text)
    except NoSuchElementException as e:
        delivery.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds  
    
    #Extracting the availability of headphones
    try:
        availability.append(driver.find_element_by_xpath("//div[@id='availability']").text)
    except NoSuchElementException as e:    
        availability.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds  
    
    #Extracting the other details available
    try:
        details=[i.text.replace('\n','---') for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")]
        other_details.append(details[0])
    except NoSuchElementException as e:     
        other_details.append("-")
    driver.implicitly_wait(3)  #Making the driver automatically wait for 3 seconds     

In [6]:
#Checking the length of data extracted
print(len(brand_name),len(product_name),len(Rating),len(no_of_ratings),len(price),len(returns),len(delivery),
      len(availability),len(other_details),len(product_url))

66 66 66 66 66 66 66 66 66 66


In [7]:
#Creating a dataframe and checking the data extracted
amazon=pd.DataFrame({'Brand Name':brand_name,'Name of Product':product_name,'Rating':Rating,'No of Ratings':no_of_ratings,
                     'Price':price,'Return/Exchange':returns,'Expected Delivery':delivery,'Availability':availability,
                     'Other details':other_details,'Product URL':product_url})
amazon

,Brand Name,Name of Product,Rating,No of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other details,Product URL
0,Fire-Boltt,Blast 1400 Over -Ear Bluetooth Wireless Head...,4.1 out of 5 stars,"3,505 ratings","₹ 2,499.00",7 Days Replacement,"Friday, April 23",In stock.,"Brand Fire-Boltt---Manufacturer Fire-Boltt, Bo...",https://www.amazon.in/gp/slredirect/picassoRed...
1,Marshall,Major III On-Ear Headphones (White),4.4 out of 5 stars,238 ratings,"₹ 4,999.00",7 Days Replacement,"Friday, April 23",In stock.,Brand Marshall---Manufacturer Marshall---Model...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Zebronics,Zeb-Lark Wireless in Ear Neckband Earphone w...,3.4 out of 5 stars,781 ratings,₹ 798.00,7 Days Replacement,"Friday, April 23",In stock.,"Brand ZEBRONICS---Manufacturer ZEBRONICS, ZEBR...",https://www.amazon.in/Zebronics-Zeb-Lark-Bluet...
3,boAt,Bassheads 900 On Ear Wired Headphones(Carbon...,4.2 out of 5 stars,"32,957 ratings",₹ 749.00,7 Days Replacement,"Friday, April 23",In stock.,Brand Boat---Manufacturer Imagine Marketing Pv...,https://www.amazon.in/Boat-BassHeads-900-Wired...
4,Boult,Audio Bass Buds Q2 Over-Ear Wired Lightweigh...,3.9 out of 5 stars,"1,901 ratings",₹ 499.00,7 Days Replacement,"Friday, April 23",In stock.,"Brand Boult Audio---Manufacturer Exotic Mile, ...",https://www.amazon.in/Boult-Audio-Wired-Lightw...
...,...,...,...,...,...,...,...,...,...,...
61,Infinity,(JBL) Zip 100 in-Ear Immersive Bass Tangle F...,3.9 out of 5 stars,"5,673 ratings",₹ 469.00,7 Days Replacement,"Friday, April 23",,Brand Infinity---Manufacturer Harman Internati...,https://www.amazon.in/Infinity-Zip-100-Immersi...
62,Lopina,SH-12 Wireless Universal Bluetooth Headphone...,3.6 out of 5 stars,162 ratings,₹ 439.00,7 Days Replacement,-,In stock.,Brand LOPINA- MASTER OF INNOVATION POWER---Man...,https://www.amazon.in/SH-12-Wireless-Universal...
63,EKSA,E900 Wired Stereo Gaming Headset-Over Ear He...,4.4 out of 5 stars,"3,572 ratings","₹ 2,299.00",7 Days Replacement,"Monday, April 26",In stock.,Brand EKSA---Manufacturer EKSA---Model E900---...,https://www.amazon.in/EKSA-Headset-Over-Headph...
64,British,Acoustics DELTA D2 B Headphone,3.9 out of 5 stars,22 ratings,"₹ 2,499.00",7 Days Replacement,"Tuesday, April 27",In stock.,Package Dimensions 24 x 22.5 x 11 cm; 272 Gram...,https://www.amazon.in/gp/slredirect/picassoRed...


In [8]:
#Saving the data into a csv file
amazon.to_csv("Amazon_HeadphonesData.csv")

In [9]:
#Closing the driver
driver.close()